In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot as plt
from sklearn.metrics import r2_score

In [ ]:
# Reading Data from Several Data Sources
df1= pd.read_csv('C:/Users/grego/Desktop/Data Science and Analytics MSc/MATH5872M Dissertation in Data Science and Analytics/Telecommunication/csv files of dataset/Data traffic across UK.csv')
df2= pd.read_csv('C:/Users/grego/Desktop/Data Science and Analytics MSc/MATH5872M Dissertation in Data Science and Analytics/Telecommunication/csv files of dataset/2021 Median weekly pay.csv')
df3= pd.read_csv('C:/Users/grego/Desktop/Data Science and Analytics MSc/MATH5872M Dissertation in Data Science and Analytics/Telecommunication/csv files of dataset/ethnic minorities.csv')
df4= pd.read_csv('C:/Users/grego/Desktop/Data Science and Analytics MSc/MATH5872M Dissertation in Data Science and Analytics/Telecommunication/csv files of dataset/GDP per head UK local authorities.csv')
df5= pd.read_csv('C:/Users/grego/Desktop/Data Science and Analytics MSc/MATH5872M Dissertation in Data Science and Analytics/Telecommunication/csv files of dataset/Land Area of UK Local Authorities.csv')
df6= pd.read_csv('C:/Users/grego/Desktop/Data Science and Analytics MSc/MATH5872M Dissertation in Data Science and Analytics/Telecommunication/csv files of dataset/Population2.csv')
df7=pd.read_csv('C:/Users/grego/Desktop/Data Science and Analytics MSc/MATH5872M Dissertation in Data Science and Analytics/Telecommunication/csv files of dataset/median age and people per square km.csv')
df8= pd.read_csv('C:/Users/grego/Desktop/Data Science and Analytics MSc/MATH5872M Dissertation in Data Science and Analytics/Telecommunication/csv files of dataset/deprivation.csv')

In [ ]:
df7

In [ ]:
df8

In [ ]:
from functools import reduce

In [ ]:
# Merge  pandas DataFrames
data_merge1 = reduce(lambda left, right:   
                     pd.merge(left , right,
                              on = ["Local Authority"]),
                     [df1, df3, df4, df5, df6, df7, df8])
data_merge1= data_merge1.set_index('Local Authority') 
data_merge1=data_merge1.reindex(columns=['Total Population','Land Area: Kilometers Square','Population Density (People per sq. km)', 'GDP per Capita',

                                         'Percentage of Ethnic Minorities', 'Median Age', 'IMD Deprivation Score', '4G Data Traffic (TB)', 'Data Traffic (TB)'])
data_merge1# Print merged DataFrame

In [ ]:
data_merge1.to_csv('complete dataset.csv')

In [ ]:
corr_matrix=data_merge1.corr() # Get correlation matrix 
corr_matrix
corr_matrix["Data Traffic (TB)"].sort_values(ascending=False) # sort in descending order to show strength of correlation 
abscorr_matrix=pd.DataFrame(abs(corr_matrix["Data Traffic (TB)"])) # Get strength of both positive and negative correlations
print(corr_matrix["Data Traffic (TB)"].sort_values(ascending=False)[2:9])# computed correlations of features to targets

In [ ]:
from pandas.plotting import scatter_matrix
attributes=["Data Traffic (TB)", "Total Population", "Population Density (People per sq. km)","IMD Deprivation Score"]
scatter_matrix(data_merge1[attributes],figsize=(20,14)) # Scatter matrix of  correlations 

In [ ]:
from pandas.plotting import scatter_matrix
from pandas.plotting import scatter_matrix
attributes2=["Data Traffic (TB)", "GDP per Capita", "Land Area: Kilometers Square", "Median Age"]
scatter_matrix(data_merge1[attributes2],figsize=(20,14)) # Scatter matrix of  correlations  

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

data_merge1.hist(bins=50,figsize=(25,20)) # Histogram of features/predictors

In [ ]:
features= data_merge1.drop(columns=[ 'Data Traffic (TB)', '4G Data Traffic (TB)'])

In [ ]:
features

In [ ]:
features.hist(bins=50,figsize=(25,20)) # Histogram of features/predictors

In [ ]:
targets= data_merge1[['Data Traffic (TB)']]

In [ ]:
targets

# Splitting Data into Train and Test

In [ ]:
from sklearn.model_selection import train_test_split

train_features, test_features = train_test_split(features, test_size= 0.3, random_state=50)
print(len(train_features))
print(len(test_features))

In [ ]:
train_targets, test_targets = train_test_split(targets, test_size= 0.3, random_state=50)
print(len(train_targets))
print(len(test_targets))

In [ ]:
train_features

In [ ]:
train_targets

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


pipeline = Pipeline([
   ('std_scaler',StandardScaler())])

cleantraining=pipeline.fit_transform(train_features)

print(cleantraining.shape)

In [ ]:
newclean=pd.DataFrame(cleantraining)

In [ ]:
data_merge1.to_csv('complete dataset.csv')

In [ ]:
EDAOS=newclean.describe()
EDAOS

In [ ]:
#EDAOS.to_csv('standatdized EDA.csv')

# Training OLS Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
lin_regressor=LinearRegression()

olsmse = cross_val_score(lin_regressor,cleantraining, train_targets, scoring='neg_mean_squared_error', cv=10)   
olsrmse_scores = np.sqrt(-olsmse)
def display_scores(scores):
    #print('Scores:',scores)
    print('Mean rmse:',scores.mean())
    #print('Standard deviation:',scores.std())
print('OLS regression model score:')
display_scores(olsrmse_scores)

In [ ]:
lin_regressor.fit(cleantraining, train_targets)
some_data = train_features.iloc[:5]
some_labels= train_targets.iloc[:5]
some_data_prepared = pipeline.transform(some_data)
print("Predictions:",lin_regressor.predict(some_data_prepared))
print("Labels: ",some_labels)

# Testing Linear Regression Model on Test Data

In [ ]:
final_model=lin_regressor
final_model.fit(cleantraining, train_targets)
test_prepared = pipeline.fit_transform(test_features)
final_predictions=final_model.predict(test_prepared)
final_mse = mean_squared_error(test_targets,final_predictions)
final_rsme = np.sqrt(final_mse)
print('Root mean squared error for test data is:', final_rsme)
print("R2 Score is:", r2_score(test_targets, final_predictions))
R2_Score= r2_score(test_targets, final_predictions)
print('Linear Regression Model has an accuracy of', R2_Score*100,'%', 'based on R2 Score')


In [ ]:
final_predictions=final_predictions.tolist() # Convert array to list

In [ ]:
plt.figure(figsize=(10,5))
plt.scatter(test_targets, final_predictions)
plt.xlabel('Actual Mobile Data Traffic (TB)', fontsize=14)
plt.ylabel('Predicted Mobile Data Traffic (TB)', fontsize=14)
plt.title('Predicted vs Actual (Test Data)')

# Feature Importance OLS Regression

In [ ]:
coefficients = lin_regressor.coef_
coefficients=coefficients.tolist() # Convert array to list
coefficients=sum(coefficients, []) # Flatten list of list to just a list

In [ ]:
feature_importances=np.abs(coefficients) # Absolute values of coefficients
num_attribs=list(train_features)
feature_importances


In [ ]:
Rank_of_features=sorted(zip(feature_importances, num_attribs), reverse= True)[:7] # Rank most important features
ranking=Rank_of_features[0][0], Rank_of_features[1][0], Rank_of_features[2][0], Rank_of_features[3][0],Rank_of_features[4][0], Rank_of_features[5][0], Rank_of_features[6][0]
ranking_list=list(ranking)
ranking_list
ranked_features=Rank_of_features[0][1],Rank_of_features[1][1],Rank_of_features[2][1],Rank_of_features[3][1],Rank_of_features[4][1],Rank_of_features[5][1],Rank_of_features[6][1]
ranked_features_list=list(ranked_features)
ranked_features_list
shortened_ranked_features_list= ['Total Population', 'Population Density', 'GDP per Capita', '% of Ethnic Minorities', 'Land Area', 'Deprivation Score', 'Median Age']
shortened_ranked_features_list

In [ ]:
fig = plt.figure(figsize = (13, 6))
def addlabels(x,y):
     for i in range(len(x)):
             plt.text(round(i),round(y[i]),round(y[i]), ha='center',bbox=dict(facecolor='orange'))
# creating the bar plot
plt.bar(shortened_ranked_features_list, ranking_list, 
        width = 0.4, align='center')
addlabels(shortened_ranked_features_list, ranking_list)
 
plt.xlabel("Features", fontsize=20)
plt.ylabel("Magnitude of Importance", fontsize=20)
plt.title("Linear Regression Feature Importance Ranking", fontsize=20)
plt.show()

In [ ]:
final_predictions=sum(final_predictions, []) # Flatten list of list to just a single list

In [ ]:
# Getting all models predicted targets for test set
pred_targets_df=pd.DataFrame({'Actual Targets':test_targets['Data Traffic (TB)'], 'Model Predicted Targets':final_predictions, 
                              '% Difference': abs((test_targets['Data Traffic (TB)']-final_predictions)/(test_targets['Data Traffic (TB)'])*100)})
sample_pred_targets=pred_targets_df[7:13]
sample_pred_targets1=pred_targets_df[1:7]
sample_pred_targets2=pred_targets_df[13:19]
sample_pred_targets3=pred_targets_df[19:25]
sample_pred_targets4=pred_targets_df[25:31]
sample_pred_targets5=pred_targets_df[31:37]
sample_pred_targets6=pred_targets_df[37:43]
sample_pred_targets7=pred_targets_df[43:49]
sample_pred_targets8=pred_targets_df[49:55]
sample_pred_targets9=pred_targets_df[55:63]

pred_targets_df

# Showing Performance of the model agaisnt Actual Values

In [ ]:
# Actual and predicted first set
plt.rcParams["figure.figsize"]=(8,5.5)
sample_pred_targets[['Actual Targets', 'Model Predicted Targets']].plot(marker='o')
plt.title('Monthly Mobile Data Traffic vs Local Authority', fontsize=16)
plt.xlabel('Local Authority', fontsize=16)
plt.ylabel('Monthly Mobile Data Traffic (TB)', fontsize=16)
plt.grid(True)
plt.show()

In [ ]:
# Actual and predicted second set
plt.rcParams["figure.figsize"]=(8,5.5)
sample_pred_targets1[['Actual Targets', 'Model Predicted Targets']].plot(marker='o')
plt.title('Monthly Mobile Data Traffic vs Local Authority', fontsize=16)
plt.xlabel('Local Authority', fontsize=16)
plt.ylabel('Monthly Mobile Data Traffic (TB)', fontsize=16)
plt.grid(True)
plt.show()

In [ ]:
# Actual and predicted third set
plt.rcParams["figure.figsize"]=(8,5.5)
sample_pred_targets2[['Actual Targets', 'Model Predicted Targets']].plot(marker='o')
plt.title('Monthly Mobile Data Traffic vs Local Authority', fontsize=16)
plt.xlabel('Local Authority', fontsize=16)
plt.ylabel('Monthly Mobile Data Traffic (TB)', fontsize=16)
plt.grid(True)
plt.show()

In [ ]:
# Actual and predicted fourth set
plt.rcParams["figure.figsize"]=(8,5.5)
sample_pred_targets3[['Actual Targets', 'Model Predicted Targets']].plot(marker='o')
plt.title('Monthly Mobile Data Traffic vs Local Authority', fontsize=16)
plt.xlabel('Local Authority', fontsize=16)
plt.ylabel('Monthly Mobile Data Traffic (TB)', fontsize=16)
plt.grid(True)
plt.show()

In [ ]:
# Actual and predicted fifth set
plt.rcParams["figure.figsize"]=(8,5.5)
sample_pred_targets4[['Actual Targets', 'Model Predicted Targets']].plot(marker='o')
plt.title('Monthly Mobile Data Traffic vs Local Authority', fontsize=16)
plt.xlabel('Local Authority', fontsize=16)
plt.ylabel('Monthly Mobile Data Traffic (TB)', fontsize=16)
plt.grid(True)
plt.show()

In [ ]:
# Actual and predicted sixth set
plt.rcParams["figure.figsize"]=(8,5.5)
sample_pred_targets5[['Actual Targets', 'Model Predicted Targets']].plot(marker='o')
plt.title('Monthly Mobile Data Traffic vs Local Authority', fontsize=16)
plt.xlabel('Local Authority', fontsize=16)
plt.ylabel('Monthly Mobile Data Traffic (TB)', fontsize=16)
plt.grid(True)
plt.show()

# Training Random Forest


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
# Using Grid Search to determine the best random forest model
param_grid  = [{'n_estimators': [10, 25, 40, 55, 70, 85, 100,115,130, 145, 160, 175],'max_features':[1,2,3,4,5,6,7]}, 
               {'bootstrap': [False],'n_estimators':[10, 25, 40, 55, 70, 85, 100, 115, 130, 145, 160, 175],'max_features':[1,2,3,4,5,6,7]}]

forest_reg=RandomForestRegressor(random_state=3)

grid_search = GridSearchCV(forest_reg, param_grid, cv=5,scoring='neg_mean_squared_error')
grid_search.fit(cleantraining, train_targets.squeeze())

In [ ]:
# Retrieving best parameters from GridSearchCV
grid_search.best_params_

In [ ]:
# Displaying best estimator
grid_search.best_estimator_

In [ ]:
# re-train random forest with best parameters gotten from grid search
bestforest_reg=grid_search.best_estimator_

In [ ]:
# Use cross validation to get rmse values for random forest model
forestscores=cross_val_score(bestforest_reg, cleantraining, train_targets.squeeze(),scoring="neg_mean_squared_error",cv=10)
forest_rmse_scores=np.sqrt(-forestscores)
print('Random Forest rmse scores:')
display_scores(forest_rmse_scores)


# Testing Random Forest Model on Test Data

In [ ]:
randomforest_reg=grid_search.best_estimator_
randomforest_reg.fit(cleantraining, train_targets.squeeze())
rftest_prepared = pipeline.fit_transform(test_features)
rffinal_predictions=randomforest_reg.predict(test_prepared)
randomforest_mse = mean_squared_error(test_targets,rffinal_predictions)
randomforest_rsme = np.sqrt(randomforest_mse)
print('Random Forest root mean squared error for test data is:', randomforest_rsme)
print("R2 Score is:", r2_score(test_targets, rffinal_predictions))
R2_Scorerf= r2_score(test_targets, rffinal_predictions)
print('Random Forest Regression Model has an accuracy of', R2_Scorerf*100,'%', 'based on R2 Score')

In [ ]:
rffinal_predictions=rffinal_predictions.tolist() # Convert array to list

In [ ]:
# Plotting model prediction vs actual 
plt.figure(figsize=(10,5))
plt.scatter(test_targets, rffinal_predictions)
plt.xlabel('Actual Data Traffic (TB)', fontsize=14)
plt.ylabel('Predicted Data Traffic (TB)', fontsize=14)
plt.title('Predicted vs Actual (Test Data)')

# Getting Feature Importance for Random Forest

In [ ]:
rffeature_importances = grid_search.best_estimator_.feature_importances_
rffeature_importances

In [ ]:
rfRank_of_features=sorted(zip(rffeature_importances,num_attribs),reverse=True)[:8]
rfRank_of_features

In [ ]:
rfranking=rfRank_of_features[0][0], rfRank_of_features[1][0], rfRank_of_features[2][0], rfRank_of_features[3][0],rfRank_of_features[4][0], rfRank_of_features[5][0], rfRank_of_features[6][0]
rfranking_list=list(rfranking)
rfranking_list
rfshortened_ranked_features_list= ['Total Population', '% of Ethnic Minorities', 'Population Density', 'Deprivation Score', 'Median Age', 'Land Area',  'GDP per Capita']
rfshortened_ranked_features_list

In [ ]:
fig = plt.figure(figsize = (14, 7))
def addlabels(x,y):
     for i in range(len(x)):
             plt.text(round(i,2),round(y[i],2),round(y[i],2), ha='center',bbox=dict(facecolor='orange'))
# creating the bar plot
plt.bar(rfshortened_ranked_features_list, rfranking_list, 
        width = 0.4, align='center')
addlabels(rfshortened_ranked_features_list, rfranking_list)
 
plt.xlabel("Features", fontsize=23)
plt.ylabel("Magnitude of Importance", fontsize=23)
plt.title("Random Forest Feature Importance Ranking", fontsize= 23)
plt.show()

# Random Forest Model Predictions for Test set

In [ ]:
# Getting all model predicted targets for test set
rfpred_targets_df=pd.DataFrame({'Actual Targets':test_targets['Data Traffic (TB)'], 'Model Predicted Targets':rffinal_predictions, 
                              '% Difference': abs((test_targets['Data Traffic (TB)']-rffinal_predictions)/(test_targets['Data Traffic (TB)'])*100)})
rfsample_pred_targets=rfpred_targets_df[7:13]
rfsample_pred_targets1=pred_targets_df[1:7]
rfsample_pred_targets2=pred_targets_df[13:19]
rfsample_pred_targets3=pred_targets_df[19:25]
rfsample_pred_targets4=pred_targets_df[25:31]
rfsample_pred_targets5=pred_targets_df[31:37]
rfsample_pred_targets6=pred_targets_df[37:43]
rfsample_pred_targets7=pred_targets_df[43:49]
rfsample_pred_targets8=pred_targets_df[49:55]
rfsample_pred_targets9=pred_targets_df[55:63]
rfpred_targets_df

In [ ]:
# Actual vs predicted first set
plt.rcParams["figure.figsize"]=(10,6.5)
rfsample_pred_targets[['Actual Targets', 'Model Predicted Targets']].plot(marker='o')
plt.title('Monthly Mobile Data Traffic vs Local Authority', fontsize=20)
plt.xlabel('Local Authority', fontsize=20)
plt.ylabel('Monthly Mobile Data Traffic (TB)', fontsize=20)
plt.grid(True)
plt.show()

In [ ]:
# Actual vs predicted second set
plt.rcParams["figure.figsize"]=(10,6.5)
rfsample_pred_targets1[['Actual Targets', 'Model Predicted Targets']].plot(marker='o')
plt.title('Monthly Mobile Data Traffic vs Local Authority', fontsize=20)
plt.xlabel('Local Authority', fontsize=20)
plt.ylabel('Monthly Mobile Data Traffic (TB)', fontsize=20)
plt.grid(True)
plt.show()

In [ ]:
# Actual vs predicted third set
plt.rcParams["figure.figsize"]=(10,6.5)
rfsample_pred_targets2[['Actual Targets', 'Model Predicted Targets']].plot(marker='o')
plt.title('Monthly Mobile Data Traffic vs Local Authority', fontsize=20)
plt.xlabel('Local Authority', fontsize=20)
plt.ylabel('Monthly Mobile Data Traffic (TB)', fontsize=20)
plt.grid(True)
plt.show()

In [ ]:
# Actual vs predicted fourth set
plt.rcParams["figure.figsize"]=(10,6.5)
rfsample_pred_targets3[['Actual Targets', 'Model Predicted Targets']].plot(marker='o')
plt.title('Monthly Mobile Data Traffic vs Local Authority', fontsize=20)
plt.xlabel('Local Authority', fontsize=20)
plt.ylabel('Monthly Mobile Data Traffic (TB)', fontsize=20)
plt.grid(True)
plt.show()